In [1]:
import os
import subprocess
from itertools import product

# Define the range of parameters
queue_depths = [1,2,4,8]
num_queues = [1, 2, 4, 8]
block_sizes = [1, 2, 4, 8]
num_blocks = [1,2,4,8]
page_sizes = [4096]
io_types = [0, 1]
io_methods = [0, 1]
num_io = 10000

# Output directory for results
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

exec_path = os.path.join(os.getcwd(), "../build/bin/block-test")
print(f"Executable path: {exec_path}")

# Function to run the command and save results
def run_command_and_save_results(exec_path, params):
    queue_depth, num_queue, block_size, num_block, page_size, io_type, io_method = params
    output_file = os.path.join(
        output_dir, 
        f"qd{queue_depth}_nq{num_queue}_bs{block_size}_nb{num_block}_ps{page_size}_it{io_type}_im{io_method}.txt"
    )

    command = [
        "sudo", exec_path,
        f"--device", "0",
        f"--controller", "0",
        f"--queue-depth", str(queue_depth),
        f"--num-queues", str(num_queue),
        f"--block-size", str(block_size),
        f"--num-blocks", str(num_block),
        f"--page-size", str(page_size),
        f"--io-type", str(io_type),
        f"--io-method", str(io_method),
        f"--num-io", str(num_io)
    ]

    try:
        with open(output_file, "w") as f:
            result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            f.write(result.stdout)
            f.write("\\n--- STDERR ---\\n")
            f.write(result.stderr)
        print(f"Saved results to {output_file}")
    except Exception as e:
        print(f"Failed to run command: {e}")
        
# Generate all combinations of parameters
parameter_combinations = product(queue_depths, num_queues, block_sizes, num_blocks, page_sizes, io_types, io_methods)

# Run the commands for each combination
for params in parameter_combinations:
    run_command_and_save_results(exec_path, params)

print("All commands executed. Results are saved in the 'results' directory.")


Executable path: /export/home1/ltarun/bam-test/scripts/../build/bin/block-test
Saved results to results/qd1_nq1_bs1_nb1_ps4096_it0_im0.txt
Saved results to results/qd1_nq1_bs1_nb1_ps4096_it0_im1.txt
Saved results to results/qd1_nq1_bs1_nb1_ps4096_it1_im0.txt
Saved results to results/qd1_nq1_bs1_nb1_ps4096_it1_im1.txt
Saved results to results/qd1_nq1_bs1_nb2_ps4096_it0_im0.txt
Saved results to results/qd1_nq1_bs1_nb2_ps4096_it0_im1.txt
Saved results to results/qd1_nq1_bs1_nb2_ps4096_it1_im0.txt
Saved results to results/qd1_nq1_bs1_nb2_ps4096_it1_im1.txt
Saved results to results/qd1_nq1_bs1_nb4_ps4096_it0_im0.txt
Saved results to results/qd1_nq1_bs1_nb4_ps4096_it0_im1.txt
Saved results to results/qd1_nq1_bs1_nb4_ps4096_it1_im0.txt
Saved results to results/qd1_nq1_bs1_nb4_ps4096_it1_im1.txt
Saved results to results/qd1_nq1_bs1_nb8_ps4096_it0_im0.txt
Saved results to results/qd1_nq1_bs1_nb8_ps4096_it0_im1.txt
Saved results to results/qd1_nq1_bs1_nb8_ps4096_it1_im0.txt
Saved results to resu

In [ ]:
import os
import re
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict

# Directory containing results
result_dir = "results"

plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)

# Define linestyles, markers, and colors
linestyle_tuple = [
    # ('loosely dotted', (0, (1, 10))),
    ('dotted', (0, (1, 1))),
    ('densely dotted', (0, (1, 1))),
    ('long dash with offset', (5, (10, 3))),
    ('loosely dashed', (0, (5, 10))),
    ('dashed', (0, (5, 5))),
    ('densely dashed', (0, (5, 1))),
    ('loosely dashdotted', (0, (3, 10, 1, 10))),
    ('dashdotted', (0, (3, 5, 1, 5))),
    ('densely dashdotted', (0, (3, 1, 1, 1))),
    ('dashdotdotted', (0, (3, 5, 1, 5, 1, 5))),
    ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
    ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))
]

marker_shapes = ['o', 's', '^', 'D', 'v', 'p', '*', 'h', 'H', '+', 'x', 'X']
colors = plt.cm.tab20.colors  # Use a colormap for distinct colors

# Generate unique combinations of line style, marker, and color
style_combinations = list(itertools.product(linestyle_tuple, marker_shapes, colors))

# Regular expressions to extract data
kernel_time_pattern = r"Kernel execution time:\s+([\d\.]+) ms"
data_transferred_pattern = r"Data transferred:\s+([\d\.]+) MB"
bandwidth_pattern = r"Bandwidth:\s+([\d\.]+) MB/s"
throughput_pattern = r"Throughput:\s+([\d\.]+) IOPS"

# Function to parse parameters from filenames
def parse_parameters(filename):
    params = {}
    for part in filename[:-4].split("_"):
        if part.startswith("qd"):
            params["queue_depth"] = int(part[2:])
        elif part.startswith("nq"):
            params["num_queues"] = int(part[2:])
        elif part.startswith("bs"):
            params["block_size"] = int(part[2:])
        elif part.startswith("nb"):
            params["num_blocks"] = int(part[2:])
        elif part.startswith("ps"):
            params["page_size"] = int(part[2:])
        elif part.startswith("it"):
            params["io_type"] = int(part[2:])
        elif part.startswith("im"):
            params["io_method"] = int(part[2:])
    return params

# Initialize results storage
results = defaultdict(list)

# Read all result files
for filename in os.listdir(result_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(result_dir, filename)
        with open(file_path, "r") as file:
            content = file.read()

            # Extract data using regex
            kernel_time_match = re.search(kernel_time_pattern, content)
            data_transferred_match = re.search(data_transferred_pattern, content)
            bandwidth_match = re.search(bandwidth_pattern, content)
            throughput_match = re.search(throughput_pattern, content)

            if kernel_time_match and data_transferred_match and bandwidth_match and throughput_match:
                params = parse_parameters(filename)
                results[(params["queue_depth"], params["num_queues"], params["block_size"], params["num_blocks"])].append({
                    "kernel_time": float(kernel_time_match.group(1)) / 1000,  # Convert to seconds
                    "data_transferred": float(data_transferred_match.group(1)),
                    "bandwidth": float(bandwidth_match.group(1)),
                    "throughput": float(throughput_match.group(1)),
                    "page_size": params.get("page_size"),
                    "io_type": params.get("io_type"),
                    "io_method": params.get("io_method")
                })

# Group and plot results
parameters = ["queue_depth", "num_queues", "block_size", "num_blocks"]
metrics = ["kernel_time", "bandwidth", "throughput", "data_transferred"]
ylabel_map = {
    "kernel_time": "Runtime (s)",
    "bandwidth": "Bandwidth (MB/s)",
    "throughput": "IOPS",
    "data_transferred": "Data Transferred (MB)"
}

for varying_param in parameters:
    fixed_params = [p for p in parameters if p != varying_param]

    # Select one of the fixed params to also vary in the plot as a line
    for secondary_varying_param in fixed_params:
        remaining_fixed_params = [p for p in fixed_params if p != secondary_varying_param]

        grouped_results = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

        for (queue_depth, num_queues, block_size, num_blocks), metrics_data_list in results.items():
            for metrics_data in metrics_data_list:
                param_values = {
                    "queue_depth": queue_depth,
                    "num_queues": num_queues,
                    "block_size": block_size,
                    "num_blocks": num_blocks
                }

                fixed_values = tuple(param_values[p] for p in remaining_fixed_params)
                varying_value = param_values[varying_param]
                secondary_varying_value = param_values[secondary_varying_param]

                for metric in metrics:
                    grouped_results[fixed_values][secondary_varying_value][metric].append((varying_value, metrics_data[metric]))

        for fixed_values, secondary_group in grouped_results.items():
            print(f"Processing fixed_values: {fixed_values}")
            print(f"Number of secondary groups: {len(secondary_group)}")

            fig, axes = plt.subplots(2, 2, figsize=(12, 10))  # Create a 2x2 grid of subplots

            for ax, metric in zip(axes.flatten(), metrics):
                style_idx = 0  # Reset style index for each metric
                for secondary_value, metrics_by_varying in secondary_group.items():
                    sorted_data = sorted(metrics_by_varying[metric], key=lambda x: x[0])
                    varying_values = [item[0] for item in sorted_data]
                    metric_values = [item[1] for item in sorted_data]

                    if not varying_values:
                        continue  # Skip if there's no data

                    # Select a unique style
                    linestyle, marker, color = style_combinations[style_idx]
                    style_idx = (style_idx + 1) % len(style_combinations)

                    # Plot with the selected style
                    ax.plot(varying_values, metric_values, label=f"{secondary_varying_param.capitalize()}={secondary_value}",
                            linestyle=linestyle[1], marker=marker, color=color)

                # Set labels and titles for each subplot
                ax.set_title(metric.replace('_', ' ').capitalize())
                ax.set_xlabel(f"{varying_param.capitalize()}")
                ax.set_ylabel(ylabel_map[metric])
                ax.grid()

                # Ensure the plot starts at (0,0)
                ax.set_xlim(left=0)
                ax.set_ylim(bottom=0)

                # Add legend to each subplot
                ax.legend()

            # Construct a cleaner title for the fixed parameters
            fixed_title_parts = [f"{p.capitalize()}={v}" for p, v in zip(remaining_fixed_params, fixed_values)]
            fixed_title = ", ".join(fixed_title_parts)

            # Set a common title for the entire figure
            fig.suptitle(f"Metrics for {fixed_title} | X: {varying_param.capitalize()} | Line: {secondary_varying_param.capitalize()}", fontsize=14)

            # Save the combined plot
            save_name_parts = [f"{p}_{v}" for p, v in zip(remaining_fixed_params, fixed_values)]
            save_name = f"combined_{varying_param}_{secondary_varying_param}_" + "_".join(save_name_parts)
            save_name = save_name.replace(" ", "_").replace("=", "")
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to fit the suptitle
            plt.savefig(f"plots/{save_name}.svg")
            plt.close()


Processing fixed_values: (1, 4)
Number of secondary groups: 3
Processing fixed_values: (2, 1)
Number of secondary groups: 3
Processing fixed_values: (1, 1)
Number of secondary groups: 3
Processing fixed_values: (4, 1)
Number of secondary groups: 3
Processing fixed_values: (1, 2)
Number of secondary groups: 3
Processing fixed_values: (2, 8)
Number of secondary groups: 3
Processing fixed_values: (1, 8)
Number of secondary groups: 3
Processing fixed_values: (4, 2)
Number of secondary groups: 3
Processing fixed_values: (2, 2)
Number of secondary groups: 3
Processing fixed_values: (4, 8)
Number of secondary groups: 3
Processing fixed_values: (2, 4)
Number of secondary groups: 3
Processing fixed_values: (4, 4)
Number of secondary groups: 3
Processing fixed_values: (1, 4)
Number of secondary groups: 3
Processing fixed_values: (1, 1)
Number of secondary groups: 3
Processing fixed_values: (2, 1)
Number of secondary groups: 3
Processing fixed_values: (4, 1)
Number of secondary groups: 3
Processi